LINE - スーパーコンピュータの予約確認リクエスト

In [ ]:
#スーパーコンプー太を利用できる時間帯は、n個の予約時間で管理されており、それぞれの枠には現在から未来にわたって旬に

# -1 if from l to r all windows are booked
# if window free, output the fastest one
# reply to all q queries

# format
#n mn\ mn m
#a1 a2 ⋯ ama_1\ a_2\ \cdots\ a_ma1​ a2​ ⋯ am​
#qqq
#l1 r1l_1\ r_1l1​ r1​
#⋮\vdots⋮
#lq rql_q\ r_qlq​ rq​

In [17]:

DATA = """
10 5
3 4 6 7 8
5
1 1
3 7
6 8
8 9
8 8
"""


# Helper for working in Colab
DATA_ITER = iter(DATA.split("\n")[1:-1])


def input():
    """
    Get next iter, simulating the input() in HackerRank
    We overwrite the built-in function input(), which should only be done in rare exceptions like this.
    """
    return next(DATA_ITER)



In [18]:
def main(n, m, q, windows, lims):

    # Naive:
    # O(l) * O(w) = O(N^2) i.e. for each check if the spots are free

    # Smart:
    # Sort limits & iterate once & resort for printing -> O (l log(l)) + O(l) + O(w) + O (l log(l)) = O (N log(N))
    # windows are already sorted
    # Sort by l 
    lims = sorted(list(enumerate(lims)), key=lambda x: x[1][0])
    to_print = []

    # Append maximum value guaranteed to be higher than all lims to windows
    windows.append(n+1)
    # Reverse windows to pop from the last element when looping, which is O(1), while the first element is O(N)
    windows = list(reversed(windows))
    w_cursor = -1
    fin_x = windows[0]
    x = windows.pop()
    popped = True
  
    for (idx, (l, r)) in lims:
        while True:
            # reached end of limit range
            if (l == r+1):
                to_print.append((idx, -1))
                break 
            # free spot before
            elif l < x:
                to_print.append((idx, l))
                break
            # need x to catch up to current pos
            elif l > x:
                # Shorten by one step
                x = windows.pop()
                popped = True
            # move cursors - not popping as next limit may interleave
            elif l == x:
                l += 1
                # Add the current x back to the windows, as it's in range
                if popped:
                    windows.append(x)
                    w_cursor -= 1
                    popped = False
                # Grab the next x
                x = windows[w_cursor]
                w_cursor -= 1
                
        # reset x & cursor if used as next limit may interleave
        if w_cursor != -1:
            w_cursor = -1
            x = windows.pop()
            popped = True

    for idx, w in sorted(to_print, key=lambda x: x[0]):
        if w:
          print(w)

if __name__ == '__main__':
    n, m = [int(x) for x in input().rstrip().split()]
    windows = [int(x) for x in input().rstrip().split()]
    assert m == len(windows)
    q = int(input().rstrip())
    
    lims = []
    for _ in range(q):
      try:
        to_append = [int(x) for x in input().rstrip().split()]
        if to_append:
            lims.append(to_append)
      except:
        continue

    main(n, m, q, windows, lims)


1
5
-1
9
-1


LINE - 迷路とマッチョ

21/21 in 1 hour (time limit was 2 hours)

In [62]:
'''
# Sample code to perform I/O:

name = input()                  # Reading input from STDIN
print('Hi, %s.' % name)         # Writing output to STDOUT

# Warning: Printing unwanted or ill-formatted data to output will cause the test cases to fail
'''

DATA = """
5 3
...
..S
...
.G.
...
"""
DATA = """
3 3
S##
###
..G
"""
DATA = """
3 3
S##
##.
#.G
"""
# Helper for working in Colab
DATA_ITER = iter(DATA.split("\n")[1:-1])


def input():
    """
    Get next iter, simulating the input() in HackerRank
    We overwrite the built-in function input(), which should only be done in rare exceptions like this.
    """
    return next(DATA_ITER)

In [63]:
def get_edges(pos, nrows, ncols):
    """Zero-indexed"""
    edges = []
    # East
    if pos[1] < (ncols - 1):
        edges.append((pos[0], pos[1]+1))
    # South
    if pos[0] < (nrows - 1):
        edges.append((pos[0]+1, pos[1]))
    # West
    if pos[1] > 0:
        edges.append((pos[0], pos[1]-1))
    # North
    if pos[0] > 0:
        edges.append((pos[0]-1, pos[1]))
    return edges

def get_neighbors(pos, nrows, ncols, wall_pos, time, wall_count):
    # Go through possible movements
    neighbors = set()
    for edge in get_edges(pos, nrows, ncols):
        is_wall = edge in wall_pos
        # Should actually update the wall positions, but as the optimal solution never goes over the same field twice, can skip it
        neighbors.add((edge, (time+1)+(wall_count+1) if is_wall else time+1, wall_count+1 if is_wall else wall_count))
    return neighbors

def solve(n, m, matrix, goal_pos, gorilla_pos, wall_pos):

    explored_vertices = set()
    tracker = {str(gorilla_pos): (gorilla_pos, 0, -1)} # (pos, time, wall_count)

    while tracker:
        # Pick best option by time, then by wall count
        pos_name = min(tracker.items(), key=lambda item: (item[1][1], item[1][2]))[0]

        pos, time, wall_count = tracker.pop(pos_name) # Remove from tracker
        explored_vertices.add(pos_name)

        # Output result if goal reached
        if pos == goal_pos:
            print(time)
            return
        
        for i, (nbor_pos, nbor_time, nbor_wall_count) in enumerate(get_neighbors(pos, n, m, wall_pos, time, wall_count)):
            nbor_pos_name = str(nbor_pos)
            # If it has previously been the cheapest option, then there cannot be a cheaper option as time / wall_count never decreases
            if nbor_pos_name in explored_vertices:
                continue
            if nbor_pos_name in tracker:
                (cur_pos, cur_time, cur_wall_count) = tracker[nbor_pos_name]
                # Worse / Equal on both - skip
                if (nbor_time >= cur_time) and (nbor_wall_count >= cur_wall_count):
                    continue
                nbor_pos_name = str(nbor_pos) + str(nbor_time) + str(nbor_wall_count) 

            tracker[nbor_pos_name] = (nbor_pos, nbor_time, nbor_wall_count)

if __name__ == '__main__':
    n, m = [int(x) for x in input().rstrip().split()]
    goal_pos = None
    gorilla_pos = None
    wall_pos = []
    matrix = []
    for row_idx in range(n):
        row = []
        for col_idx, char in enumerate(input().rstrip()):
            if char == "G":
                goal_pos = (row_idx, col_idx)
            elif char == "S":
                gorilla_pos = (row_idx, col_idx)
            elif char == "#":
                wall_pos.append((row_idx, col_idx))
            row.append(char)
        matrix.append(row)

    solve(n, m, matrix, goal_pos, gorilla_pos, wall_pos)

5
